In [ ]:

import psycopg2

def get_connection():
    return psycopg2.connect(
        dbname="platform", 
        user="postgres", 
        password="password", 
        host="127.0.0.1", 
        port="5432"
    )

In [ ]:
connection = get_connection()
with connection.cursor() as cursor:
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS bank (id SERIAL PRIMARY KEY, account VARCHAR(255), balance FLOAT);
        """
    )

connection.commit()
connection.close()

In [ ]:
connection = get_connection()
with connection.cursor() as cursor:
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS hospital (id SERIAL PRIMARY KEY, duty Boolean);
        """
    )
    cursor.execute("INSERT INTO hospital (duty) VALUES (True);")
    cursor.execute("INSERT INTO hospital (duty) VALUES (True);")
        
connection.commit()
connection.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Alice', 100);")
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Bob', 100);")

connection.commit()
connection.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = 100 WHERE account = 'Alice';")
    cursor.execute("UPDATE bank SET balance = 100 WHERE account = 'Bob';")


connection.commit()
connection.close()

# Read Uncommitted:
Prevents: None
Allows: Dirty read, non-repeatable read, phantom read, lost update

# Read Committed:
Prevents: Dirty read
Allows: Non-repeatable read, phantom read, lost update

# Repeatable Read:
Prevents: Dirty read, non-repeatable read
Allows: Phantom read (not in PG), lost update

# non-repeatable read

In [ ]:
connection1 = get_connection()
connection1.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ)

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
connection1.commit()
connection1.close()

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = 60 WHERE account = 'Alice';")

connection.commit()
connection.close()

# Phantom read

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Dan', 100);")

connection.commit()
connection.close()

# Lost Update:
Two or more transactions read the same data and then update it based on the read value. The final update will overwrite the changes made by the other transactions, leading to lost updates.

In [ ]:
connection1 = get_connection()
connection1.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ)
connection2 = get_connection()
connection2.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ)


In [ ]:
connection1.commit()
connection1.close()

In [ ]:
connection2.commit()
connection2.close()

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = balance - 50 WHERE account = 'Alice';")
    cursor.execute("UPDATE bank SET balance = balance + 50 WHERE account = 'Bob';")
    

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("UPDATE bank SET balance = balance + 100 WHERE account = 'Alice';")
    cursor.execute("UPDATE bank SET balance = balance - 100 WHERE account = 'Bob';")

# Serializable:
Prevents: Dirty read, non-repeatable read, phantom read, lost update
Provides the highest level of isolation, effectively preventing all the mentioned anomalies.

# Write Skew:
Two transactions read overlapping sets of data and then update non-overlapping sets of data. The final state of the database may violate integrity constraints because the transactions were unaware of each other's updates.

In [ ]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM hospital;")
    for row in cursor.fetchall():
        print(row)

connection.commit()
connection.close()

In [ ]:
connection1 = get_connection()
connection1.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE)
connection2 = get_connection()
connection2.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE)


In [ ]:
connection1.commit()
connection1.close()

In [ ]:
connection2.commit()
connection2.close()

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM hospital;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("SELECT * FROM hospital;")
    for row in cursor.fetchall():
        print(row)

In [ ]:
with connection1.cursor() as cursor:
    cursor.execute("UPDATE hospital SET duty = false WHERE id = 1;")

In [ ]:
with connection2.cursor() as cursor:
    cursor.execute("UPDATE hospital SET duty = false WHERE id = 2;")